In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../data/combined.xlsx')
weights = pd.read_excel('../data/weights.xlsx')
data.head()

,stay_id,subject_id,charttime,urineoutput,creatinine,CRRT_Status
0,32610785,10002348,2112-12-01 04:55:00,50,0.8,0
1,32610785,10002348,2112-12-01 10:00:00,175,NaN,0
2,32610785,10002348,2112-12-01 12:00:00,100,NaN,0
3,32610785,10002348,2112-12-02 12:00:00,50,NaN,0
4,32610785,10002348,2112-12-02 16:00:00,100,NaN,0


In [3]:
weights

,subject_id,max
0,10001884,65.000000
1,10002155,54.000000
2,10002348,41.600000
3,10002428,55.000000
4,10002495,64.166667
5,10003400,99.600000


In [4]:
weights.rename(columns={'max':'weight'}, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   stay_id      1150 non-null   int64  
 1   subject_id   1150 non-null   int64  
 2   charttime    1150 non-null   object 
 3   urineoutput  1150 non-null   int64  
 4   creatinine   83 non-null     float64
 5   CRRT_Status  1150 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 54.0+ KB


In [6]:
# convert charttime to datetime
data['charttime'] = pd.to_datetime(data['charttime'])
data.sort_values(by=['stay_id', 'charttime'], inplace=True)

In [7]:
# merge weights
data = pd.merge(data, weights, on='subject_id', how='left')
data

,stay_id,subject_id,charttime,urineoutput,creatinine,CRRT_Status,weight
0,31090461,10002155,2130-09-24 04:00:00,350,2.5,0,54.0
1,31090461,10002155,2130-09-24 05:00:00,45,NaN,0,54.0
2,31090461,10002155,2130-09-24 06:00:00,50,NaN,0,54.0
3,31090461,10002155,2130-09-24 07:00:00,25,NaN,0,54.0
4,31090461,10002155,2130-09-24 08:00:00,80,NaN,0,54.0
...,...,...,...,...,...,...,...
1145,38875437,10002428,2156-04-26 10:00:00,100,NaN,0,55.0
1146,38875437,10002428,2156-04-26 12:00:00,80,NaN,0,55.0
1147,38875437,10002428,2156-04-26 14:00:00,100,NaN,0,55.0
1148,38875437,10002428,2156-04-26 16:00:00,450,NaN,0,55.0


## Add Non-AKI Patient

In [8]:
stay_id = 16920624
# create np.array for 3 days of hourly charttimes with random start date
charttimes = pd.date_range(start='2019-01-01', periods=72, freq='H')

urine_output_values = np.random.randint(80, 160, size=72)
# round values to nearest 5
urine_output_values = np.round(urine_output_values / 5) * 5
urine_output_values

array([ 80., 140., 120., 155., 130.,  90., 115., 100., 160., 100.,  95.,
       115., 135., 150.,  85., 110., 110., 150., 160., 135., 130., 130.,
       120.,  95.,  80., 130.,  90.,  85., 140.,  85., 125.,  95., 130.,
       140.,  85., 150., 155., 155., 135., 130.,  90., 145., 155.,  85.,
       160., 100., 125.,  90., 100., 115., 145., 145., 135., 115.,  85.,
       125., 150.,  80.,  90., 145., 145., 155.,  90.,  95., 150., 110.,
       135., 125., 130., 145., 125., 140.])

In [9]:
# combine to dataframe
non_aki_patient = pd.DataFrame({'stay_id': stay_id,
                                'charttime': charttimes,
                                'urineoutput': urine_output_values,
                                'creatinine': 0.5,
                                'CRRT_Status': 0})
non_aki_patient

,stay_id,charttime,urineoutput,creatinine,CRRT_Status
0,16920624,2019-01-01 00:00:00,80.0,0.5,0
1,16920624,2019-01-01 01:00:00,140.0,0.5,0
2,16920624,2019-01-01 02:00:00,120.0,0.5,0
3,16920624,2019-01-01 03:00:00,155.0,0.5,0
4,16920624,2019-01-01 04:00:00,130.0,0.5,0
...,...,...,...,...,...
67,16920624,2019-01-03 19:00:00,125.0,0.5,0
68,16920624,2019-01-03 20:00:00,130.0,0.5,0
69,16920624,2019-01-03 21:00:00,145.0,0.5,0
70,16920624,2019-01-03 22:00:00,125.0,0.5,0


In [10]:
# add to data
data = pd.concat([data, non_aki_patient], ignore_index=True)
data

,stay_id,subject_id,charttime,urineoutput,creatinine,CRRT_Status,weight
0,31090461,10002155.0,2130-09-24 04:00:00,350.0,2.5,0,54.0
1,31090461,10002155.0,2130-09-24 05:00:00,45.0,NaN,0,54.0
2,31090461,10002155.0,2130-09-24 06:00:00,50.0,NaN,0,54.0
3,31090461,10002155.0,2130-09-24 07:00:00,25.0,NaN,0,54.0
4,31090461,10002155.0,2130-09-24 08:00:00,80.0,NaN,0,54.0
...,...,...,...,...,...,...,...
1217,16920624,NaN,2019-01-03 19:00:00,125.0,0.5,0,NaN
1218,16920624,NaN,2019-01-03 20:00:00,130.0,0.5,0,NaN
1219,16920624,NaN,2019-01-03 21:00:00,145.0,0.5,0,NaN
1220,16920624,NaN,2019-01-03 22:00:00,125.0,0.5,0,NaN


In [11]:
data.rename(columns={'creatinine':'creat', 'CRRT_Status': 'crrt_status'}, inplace=True)

### Deindentification

In [12]:
# drop subject_id
data.drop(columns=['subject_id'], inplace=True)
# map stay ids to random 8 digits numbers
stay_ids = data['stay_id'].unique()
# new mapping
stay_id_map = {stay_id: np.random.randint(10000000, 99999999) for stay_id in stay_ids}

# replace stay_ids in data
data['stay_id'] = data['stay_id'].map(stay_id_map)

In [13]:
stay_id_map

{31090461: 30748056,
 32128372: 78990311,
 32610785: 93711503,
 33685454: 41903507,
 33987268: 12213770,
 35479615: 25077541,
 36753294: 33810293,
 37510196: 91980666,
 38383343: 48440864,
 38875437: 95982129,
 16920624: 50523246}

In [14]:
from datetime import timedelta
import random
# subtract random date from charttime
# Define a function to generate a random timedelta
def random_timedelta():
    return timedelta(days=random.randint(1, 365))

# Group by 'stay_id' and apply the random_timedelta function to each group
data['random_timedelta'] = data.groupby('stay_id')['charttime'].transform(lambda x: random_timedelta())

# Subtract the random timedelta from 'charttime' to get deidentified dates
data['charttime'] = data['charttime'] - data['random_timedelta']

# Drop the intermediate column 'random_timedelta' if not needed
data.drop(columns=['random_timedelta'], inplace=True)

In [15]:
data

,stay_id,charttime,urineoutput,creat,crrt_status,weight
0,30748056,2129-10-18 04:00:00,350.0,2.5,0,54.0
1,30748056,2129-10-18 05:00:00,45.0,NaN,0,54.0
2,30748056,2129-10-18 06:00:00,50.0,NaN,0,54.0
3,30748056,2129-10-18 07:00:00,25.0,NaN,0,54.0
4,30748056,2129-10-18 08:00:00,80.0,NaN,0,54.0
...,...,...,...,...,...,...
1217,50523246,2018-02-28 19:00:00,125.0,0.5,0,NaN
1218,50523246,2018-02-28 20:00:00,130.0,0.5,0,NaN
1219,50523246,2018-02-28 21:00:00,145.0,0.5,0,NaN
1220,50523246,2018-02-28 22:00:00,125.0,0.5,0,NaN


In [16]:
# subtract or add either 5 or ten randomly from urine output

def random_value():
    return random.choice([5, 10, -5, -10])

data["random_urineoutput"] = data.groupby('stay_id')['urineoutput'].transform(lambda x: random_value())
data['urineoutput'] = data['urineoutput'] + data['random_urineoutput']
data

,stay_id,charttime,urineoutput,creat,crrt_status,weight,random_urineoutput
0,30748056,2129-10-18 04:00:00,345.0,2.5,0,54.0,-5
1,30748056,2129-10-18 05:00:00,40.0,NaN,0,54.0,-5
2,30748056,2129-10-18 06:00:00,45.0,NaN,0,54.0,-5
3,30748056,2129-10-18 07:00:00,20.0,NaN,0,54.0,-5
4,30748056,2129-10-18 08:00:00,75.0,NaN,0,54.0,-5
...,...,...,...,...,...,...,...
1217,50523246,2018-02-28 19:00:00,115.0,0.5,0,NaN,-10
1218,50523246,2018-02-28 20:00:00,120.0,0.5,0,NaN,-10
1219,50523246,2018-02-28 21:00:00,135.0,0.5,0,NaN,-10
1220,50523246,2018-02-28 22:00:00,115.0,0.5,0,NaN,-10


In [17]:
# convert all negative values to positive in all numeric cols
data[data._get_numeric_data().columns] = data[data._get_numeric_data().columns].abs()


In [18]:
data.to_csv('../data/test_human.xlsx', index=False)

## Prepare Test Data for Machine

In [19]:
test_machine_urineoutput = data.loc[:, ['stay_id', 'charttime', 'urineoutput']].copy()
test_machine_urineoutput.to_csv('../data/test_machine_urineoutput.csv', index=False)

test_machine_creatinine = data.loc[:, ['stay_id', 'charttime', 'creat']].copy()
test_machine_creatinine.dropna(inplace=True)
test_machine_creatinine.to_csv('../data/test_machine_creatinine.csv', index=False)
test_machine_crrt = data.loc[:, ['stay_id', 'charttime', 'crrt_status']].copy()
test_machine_crrt.dropna(inplace=True)
test_machine_crrt.to_csv('../data/test_machine_crrt.csv', index=False)
test_machine_weights = data.loc[:, ['stay_id', 'weight']].copy()
test_machine_weights.drop_duplicates(inplace=True)
test_machine_weights.to_csv('../data/test_machine_weights.csv', index=False)

In [20]:
first_day = data.groupby('stay_id')['charttime'].min().reset_index()
last_day = data.groupby('stay_id')['charttime'].max().reset_index()
los = last_day['charttime'] - first_day['charttime']
los

0     4 days 19:55:00
1    10 days 18:16:00
2     3 days 17:00:00
3     4 days 23:27:00
4     5 days 23:39:00
5    15 days 12:00:00
6     2 days 23:00:00
7    12 days 16:37:00
8     8 days 16:00:00
9     6 days 06:05:00
10    6 days 23:12:00
Name: charttime, dtype: timedelta64[ns]